# IMPORT

In [1]:
import pandas as pd
from data_manager import DataManager


In [2]:
%load_ext autoreload
%autoreload 2

# DATA ANALYSIS

To view the data, launch the dashboard, which offers numerous data visualizations.

In [3]:
file_path = 'EGB_historical_yield.csv'
data_manager = DataManager(file_path)


In [4]:
data_manager.data.shape

(2110, 21)

In [5]:
data_manager.data.head()

,Germany_5y,Germany_10y,Germany_30y,Belgium_5y,Belgium_10y,Belgium_30y,Austria_5y,Austria_10y,Austria_30y,Netherlands_5y,...,Netherlands_30y,France_5y,France_10y,France_30y,Spain_5y,Spain_10y,Spain_30y,Italy_5y,Italy_10y,Italy_30y
date,,,,,,,,,,,,,,,,,,,,,
2016-01-15,-16.4,53.9,131.2,-2.1,83.8,189.7,-8,77.7,160.4,-10.4,...,143.6,-10.4,66.1,143.6,65.4,175.0,291.2,56.1,156.4,264.9
2016-01-18,-16.6,53.6,131.2,-3.3,83.5,190.0,-8,77.4,160.7,-10.6,...,143.9,-10.6,66.0,143.9,64.0,174.0,291.5,56.6,156.4,266.9
2016-01-19,-16.7,54.8,132.2,-4.1,84.0,189.9,-8,78.3,161.8,-11.3,...,145.1,-11.3,66.1,145.1,60.5,170.1,288.2,55.3,155.7,265.2
2016-01-20,-19.8,48.1,122.5,-6.0,96.3,184.2,-11,73.2,153.1,-13.8,...,135.9,-13.8,60.9,135.9,65.7,178.2,296.7,62.2,164.9,275.9
2016-01-21,-24.0,44.9,121.4,-9.8,91.5,180.5,-16,69.6,153.0,-18.2,...,134.8,-18.2,57.0,134.8,60.2,171.9,291.5,54.4,156.1,269.5


In [6]:
data_manager.data.dtypes

Germany_5y         float64
Germany_10y        float64
Germany_30y        float64
Belgium_5y         float64
Belgium_10y        float64
Belgium_30y        float64
Austria_5y           int64
Austria_10y        float64
Austria_30y        float64
Netherlands_5y     float64
Netherlands_10y    float64
Netherlands_30y    float64
France_5y          float64
France_10y         float64
France_30y         float64
Spain_5y           float64
Spain_10y          float64
Spain_30y          float64
Italy_5y           float64
Italy_10y          float64
Italy_30y          float64
dtype: object

For the Austria 5-year rates, we only have `int64` data types, which likely indicates a lack of precision in this measurement compared to others. Given 2110 rows, the probability of having only integer numbers is virtually zero.

In [7]:
data_manager.data.isnull().sum().sort_values().head(3)

Germany_5y    0
Italy_5y      0
Spain_30y     0
dtype: int64

There are no missing data, which is good news.

### Données économiques que l'on pourrait rajouter pour aller plus loin :

- Inflation
- Chômage
- PIB (Produit Intérieur Brut)
- Indice des prix à la consommation
- Indice des prix à la production (IPP)
- Balance commerciale
- Indices boursiers
- Indicateurs de sentiment de marché
